# JAXの乱数生成
> JAXにおける乱数生成についてサンプルコードを交えて解説します。

- toc: true 
- badges: true
- comments: true
- categories: [Python, JAX, DeepLearning]
- image: images/jax-samune.png

JAX流行ってますね。JAXについての詳しい説明は、[たくさんの記事](https://www.google.com/search?q=jax%E3%81%A8%E3%81%AF)や[https://github.com/google/jax](https://github.com/google/jax)　を参照していただくとして、JAXの乱数生成について勉強してみようと思います。

# Numpyにおける乱数の再現性の確保
さて、JAXはNumpyをとても意識して作られたライブラリですが、乱数周りに関しては大きく異なる点があります。
まずは, Numpyの例を見てみます。

In [91]:
import numpy as np

In [92]:
# numpy 
x = np.random.rand()
print('x:', x)

x: 0.26455561210462697


In [93]:
for i in range(10):
    x = np.random.rand()
    print('x:', x)

x: 0.7742336894342167
x: 0.45615033221654855
x: 0.5684339488686485
x: 0.018789800436355142
x: 0.6176354970758771
x: 0.6120957227224214
x: 0.6169339968747569
x: 0.9437480785146242
x: 0.6818202991034834
x: 0.359507900573786


バラバラの結果が出てきました。これを固定するには、このようなコードを書きます。

In [94]:
for i in range(10):
    np.random.seed(0)
    x = np.random.rand()
    print('x:', x)

x: 0.5488135039273248
x: 0.5488135039273248
x: 0.5488135039273248
x: 0.5488135039273248
x: 0.5488135039273248
x: 0.5488135039273248
x: 0.5488135039273248
x: 0.5488135039273248
x: 0.5488135039273248
x: 0.5488135039273248


ところでnumpyでは、`np.random.get_state()` で乱数生成器の状態が確認できます。

In [95]:
np.random.seed(0)
state = np.random.get_state()
print(state[0])
print('[', *state[1][:10], '...')
print(*state[1][-10:], ']')

MT19937
[ 0 1 1812433255 1900727105 1208447044 2481403966 4042607538 337614300 3232553940 1018809052 ...
2906783932 3668048733 2030009470 1910839172 1234925283 3575831445 123595418 2362440495 3048484911 1796872496 ]


In [96]:
np.random.seed(20040304)
state = np.random.get_state()
print(state[0])
print('[', *state[1][:10], '...')
print(*state[1][-10:], ']')

MT19937
[ 20040304 3876245041 2868517820 934780921 2883411521 496831348 4198668490 1502140500 1427494545 3747657433 ...
744972032 1872723303 3654422950 1926579586 2599193113 3757568530 3621035041 2338180567 2885432439 2647019928 ]


逆に言えば、Numpyの乱数生成はグローバルな一つの状態に依存しています。このことは次のような弊害を生みます。

# 並列実行と実行順序、再現性 
簡単なゲームを作ってみます。　
関数`a`, `b`が乱数を生成するので、大きい数を返した方が勝ちというゲームです。

In [97]:
a = lambda : np.random.rand()
b = lambda : np.random.rand()

def battle():
    if a() > b():
        return 'A'
    else:
        return 'B'

In [98]:
for i in range(10):
    print('winner is',  battle(), '!')

winner is B !
winner is A !
winner is B !
winner is A !
winner is A !
winner is A !
winner is B !
winner is B !
winner is B !
winner is A !


また実行すれば、結果は変化します。

In [99]:
for i in range(10):
    print('winner is',  battle(), '!')

winner is B !
winner is A !
winner is A !
winner is B !
winner is B !
winner is B !
winner is A !
winner is A !
winner is A !
winner is B !


ではこの結果の再現性を持たせるにはどうすればいいでしょうか。簡単な例はこうなります。

In [100]:
res1 = []
np.random.seed(0)
for i in range(10):
    res1.append(battle())

# もう一回

res2 = []
np.random.seed(0)
for i in range(10):
    res2.append(battle())

In [101]:
print('1 | 2')
print('=====')
for i in range(10):
    print(res1[i], '|', res2[i])

1 | 2
=====
B | B
A | A
B | B
B | B
A | A
A | A
B | B
B | B
B | B
B | B


というわけで同じ結果が得られました。しかし、この結果には落とし穴があります。
関数`battle`の動作をもう少し詳しく確認してみましょう。
`a`と`b`が呼び出されるタイミングを確認してみます。

In [102]:
def a():
    print('a is called!')
    return np.random.rand()

def b():
    print('b is called!')
    return np.random.rand()

In [103]:
for i in range(5):
    battle()
    print('======')

a is called!
b is called!
a is called!
b is called!
a is called!
b is called!
a is called!
b is called!
a is called!
b is called!


このように、aはbより常に先に呼び出されます。ここまでだと何の問題もないように見えますが、実際にはそうではありません。
このコードを高速に動作させたい、つまり並列化を行う時にはどうなるでしょうか。
関数`a`, `b`に依存関係はありませんから、これらを並列に動作させても問題ないように感じます。
ですが、実際には `a`, `b`が返す関数は呼び出し順序に依存しています！従って、このままではせっかく`np.random.seed`をしても意味がなくなってしまいます。

# JAXの乱数生成

では、JAXにおける乱数生成を確認してみます。
先ほどまでで述べたように、次のような条件を満たす乱数生成器を実装したいです。

- 再現性があること
- 並列化できること

これらを実現するために、JAXでは<b>key</b>という概念が用いられます。

In [104]:
key = jax.random.PRNGKey(0)
key

DeviceArray([0, 0], dtype=uint32)

keyは単に二つの実数値からなるオブジェクトで、これを用いることによって、JAXでは乱数を生成します。

In [105]:
jax.random.normal(key)

DeviceArray(-0.20584235, dtype=float32)

そして、keyが同じであれば同じ値が生成されます。

In [106]:
print(key, jax.random.normal(key))
print(key, jax.random.normal(key))
print(key, jax.random.normal(key))
print(key, jax.random.normal(key))
print(key, jax.random.normal(key))
print(key, jax.random.normal(key))
print(key, jax.random.normal(key))
print(key, jax.random.normal(key))
print(key, jax.random.normal(key))

[0 0] -0.20584235
[0 0] -0.20584235
[0 0] -0.20584235
[0 0] -0.20584235
[0 0] -0.20584235
[0 0] -0.20584235
[0 0] -0.20584235
[0 0] -0.20584235
[0 0] -0.20584235


とはいえこれだけだとひとつの数字しか得ることができません。もっとたくさんの乱数が欲しくなった際には、`jax.random.split`を用います。

In [107]:
key1, key2 = jax.random.split(key)
print(key, '->', key1, key2)

[0 0] -> [4146024105  967050713] [2718843009 1272950319]


`jax.random.split`によって、ひとつのkeyから2つのkeyが作り出されます。
このkeyによって、また新しい乱数を生み出します。　

ちなみに、この二つのkeyは等価ですが、慣例的に二つ目を新しい乱数生成につかい、一つ目はまた新しいkeyを使うために用いられるようです。(以下のコードを参照)

In [108]:
# 慣例的に二つ目をsub_keyとして新しい乱数生成に、一つ目をまた新しい乱数を作るために使用する(下のように書くことでsplit元の古いkeyも削除できる。keyが残ると誤って同じ乱数を作ってしまうので注意が必要。)
key, sub_key = jax.random.split(key)
key, subsub_key  = jax.random.split(key)

また、同じkeyから分割されたkeyは、常に等しくなります。

In [109]:
def check_split(seed):
    key = jax.random.PRNGKey(seed)
    key, sub_key = jax.random.split(key)
    print(key, '->', key, sub_key)

In [110]:
check_split(0)
check_split(0)
check_split(0)
print('=============================================================================')
check_split(2004)
check_split(2004)
check_split(2004)

[4146024105  967050713] -> [4146024105  967050713] [2718843009 1272950319]
[4146024105  967050713] -> [4146024105  967050713] [2718843009 1272950319]
[4146024105  967050713] -> [4146024105  967050713] [2718843009 1272950319]
[2965909967 2346697052] -> [2965909967 2346697052] [2813626588  818499380]
[2965909967 2346697052] -> [2965909967 2346697052] [2813626588  818499380]
[2965909967 2346697052] -> [2965909967 2346697052] [2813626588  818499380]


また、一度に何個にもsplitできます。例えば1つのkeyから次のようにして10個のkeyを得ることができます。


In [111]:
# 何個にもsplitできる。
key = jax.random.PRNGKey(0)
key, *sub_keys = jax.random.split(key, num=10)

In [112]:
key

array([3668660785,  713825972], dtype=uint32)

In [113]:
sub_keys

[array([1185646547, 2092858387], dtype=uint32),
 array([4260797006,  129535844], dtype=uint32),
 array([ 928977296, 1618649917], dtype=uint32),
 array([2708837749, 4129373854], dtype=uint32),
 array([ 652965180, 3955248629], dtype=uint32),
 array([1312337421, 1285539814], dtype=uint32),
 array([2974568872, 3669116123], dtype=uint32),
 array([1997906629, 3379841639], dtype=uint32),
 array([4278014892, 1203387755], dtype=uint32)]

# sequential-equivalent

Numpyではsequential-equivalentが保障されています。(適切な訳語がわからない)

簡単にいうと、まとめてN個の乱数を取得することと、ひとつひとつ乱数を取得して連結したものは等価である、ということが保障されています。(以下のコードを見るとわかりやすいです)

In [114]:
# ひとつずつ
np.random.seed(0)
print(np.array([np.random.rand() for i in range(10)]))

print('================================================')

# まとめて 
np.random.seed(0)
print(np.random.rand(10))

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152]
[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152]


ところがJAXではその限りではありません。JAXで10個の配列を取得する方法としては、

- keyを10個用意する
- ひとつのkeyから10個作るということが考えられます。

In [115]:
# やり方 1: keyを10個用意
key = jax.random.PRNGKey(0)
key, *sub_keys = jax.random.split(key, 11)
print(np.array([jax.random.normal(sub_key) for sub_key in sub_keys]))

[-1.3700832  -1.6277806   1.2452871  -1.0201586   0.80342007 -1.5052081
 -1.2988805   0.3053512  -0.22334994  1.1694573 ]


In [116]:
# やり方 2: ひとつのkeyから10個作る
key = jax.random.PRNGKey(0)
print(np.array(jax.random.normal(key, shape=(10,))))

[-0.372111    0.2642311  -0.18252774 -0.7368198  -0.44030386 -0.15214427
 -0.6713536  -0.59086424  0.73168874  0.56730247]


しかし、見ての通り生成される乱数は異なっています。(JAXではsequential-equivalentは保障されません。)
この理由として、

> As in NumPy, JAX's random module also allows sampling of vectors of numbers. However, JAX does not provide a sequential equivalence guarantee, because doing so would interfere with the vectorization on SIMD hardware

が[ドキュメント](https://colab.research.google.com/github/google/jax/blob/main/docs/jax-101/05-random-numbers.ipynb#scrollTo=Fhu7ejhLB4R_)で挙げられています。
どういうことだってばよ。

[design_note](https://github.com/google/jax/blob/main/design_notes/prng.md#design)